In [1]:
from tensorflow.keras.layers import Input, Conv2D, Dropout, Flatten, Dense, MaxPool2D, Activation, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import cv2 as cv
import random
import os

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1

KeyboardInterrupt: 

In [ ]:
def read_image(path, images, labels, filenames):
    
    for root, dirs, files in os.walk(path):

        for name in dirs:
            
            direct = os.path.join(path, name)
            
            for filename in os.listdir(direct):
                
                img = cv.imread(os.path.join(path + "/" + name, filename))
                labels.append(name)
                img = cv.resize(img, Image_Size)
                images.append(img)
                filenames.append(name + '/' + filename)

In [ ]:
def create_dataframe(filenames, labels):
    
    df = pd.DataFrame({
        
    'filename' : filenames,
    'category' : labels

    })
    
    return df

In [ ]:
def draw_countplot(dataframe, y_axis, title):
    
    sns.countplot(data=dataframe, y = y_axis, width=0.3, palette = 'Set2')
    plt.title(title, weight='black')
    plt.show()

In [ ]:
Image_Width=250   #600
Image_Height=250  #400
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size = 32

In [ ]:
train_filenames = []
train_images = []
train_labels = []
train_path = './DermMel/train_sep/'

read_image(path=train_path, images=train_images, labels=train_labels, filenames=train_filenames)

### Train Data

In [ ]:
train_df = create_dataframe(filenames=train_filenames, labels=train_labels)
train_df

In [ ]:
draw_countplot(dataframe=train_df, y_axis='category', title='Train Data')

In [ ]:
test_filenames = []
test_images = []
test_labels = []
test_path = './DermMel/test/'

read_image(path=test_path, images=test_images, labels=test_labels, filenames=test_filenames)

### Test Data

In [ ]:
test_df = create_dataframe(filenames=test_filenames, labels=test_labels)
test_df

In [ ]:
draw_countplot(dataframe=test_df, y_axis='category', title='Test Data')

In [ ]:
#preprocessing
train_datag=ImageDataGenerator(rescale=1./ 255)#rescalling

train_dataset=train_datag.flow_from_dataframe(dataframe=train_df, #data_frame of data
                                              directory=train_path, #path
                                              x_col="filename",
                                              y_col="category",
                                              target_size=Image_Size, #resize image it doesnt mean converting into 2d we can give only height and breadth
                                              batch_size=32, #by default 32
                                              # validate_filenames= False
                                             )

In [ ]:
test_datag=ImageDataGenerator(rescale=1./255)
test_dataset=test_datag.flow_from_dataframe(dataframe=test_df,
                                            directory=test_path,
                                            x_col="filename",
                                            y_col="category",
                                            target_size=Image_Size,
                                            batch_size=32,
                                            validate_filenames=False
                                           )

In [ ]:
train_dataset.class_indices #gives the class of target values

In [ ]:
train_dataset[0][0].shape# 0 batch# 32 batches 128-h 128-w,3d image

In [ ]:
model=Sequential([
    Input(shape=(Image_Width, Image_Height, Image_Channels)),
    #con layer1
    Conv2D(filters=16 ,kernel_size=(3,3), strides=(1,1), padding="valid", activation="relu"),#no.of kernels,k.size,strides,padding=same keeps input ouput same size valid means no padding
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.2),
    
    #conv layer-2
    Conv2D(filters=32, kernel_size=(3,3),activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.2),
    
    
    #conv layer-3
    Conv2D(filters=32, kernel_size=(3,3),activation="relu"),
    BatchNormalization(),    
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.2),
    
    #FLATTENNING
    Flatten(),
    
    #NN
    Dense(units=64,activation="relu"),
    Dense(units=32,activation="relu"),
    
    
    #output layer
    Dense(units=2,activation="softmax")
    
])
model.summary()

In [ ]:
model.compile(loss="CategoricalCrossentropy",optimizer="adam",metrics=["accuracy", Precision()])

In [ ]:
history = model.fit(train_dataset, epochs=29, validation_data=test_dataset)

In [ ]:
model.save_weights("melanoma-classifier.h5")
print("Saved model to disk")

In [ ]:
validation_filenames = []
validation_images = []
validation_labels = []
validation_path = './DermMel/valid/'

read_image(path=validation_path, images=validation_images, labels=validation_labels, filenames=validation_filenames)

### Validation Data

In [ ]:
val_df = create_dataframe(filenames=validation_filenames, labels=validation_labels)
val_df

In [ ]:
draw_countplot(dataframe=val_df, y_axis='category', title='Validation Data')

In [ ]:
val_datag=ImageDataGenerator(rescale=1./255)
val_dataset=val_datag.flow_from_dataframe(dataframe=val_df,
                                            directory=validation_path,
                                            x_col="filename",
                                            y_col="category",
                                            target_size=Image_Size,
                                            batch_size=32,
                                            validate_filenames=False
                                           )

In [ ]:
test_pred=model.predict(test_dataset)

In [ ]:
test_df["predict"]=np.argmax(test_pred,axis=1)

In [ ]:
test_df["predict_c"]=test_df["predict"].map({0:"NotMelanoma",1:"Melanoma"})

In [ ]:
test_df

In [ ]:
sns.countplot(data=test_df, x='category')

In [ ]:
sns.countplot(data=test_df, x='predict_c')

In [ ]:
val_pred=model.predict(val_dataset)

In [ ]:
val_df["predict"]=np.argmax(val_pred,axis=1)

In [ ]:
val_df["predict_c"]=val_df["predict"].map({0:"NotMelanoma",1:"Melanoma"})

In [ ]:
test_df

In [ ]:
sns.countplot(data=val_df, x='category')

In [ ]:
sns.countplot(data=val_df, x='predict_c')

In [ ]:
sns.heatmap(history)